In [189]:
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import gensim
from IPython.display import display, HTML

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


 **Load data file**

In [0]:
data = pd.read_csv('amazon_reviews.txt', sep="\t",  error_bad_lines=False)
x_raw = data['REVIEW_TEXT']
x_raw = data['REVIEW_TEXT'].values.tolist()
x_raw = [a.replace("<br />", "") for a in x_raw]
x_raw = [a.replace("good", "") for a in x_raw]
x_raw = [a.replace("great", "") for a in x_raw]
x_raw = [a.replace("use", "") for a in x_raw]
x_raw = [a.replace("just", "") for a in x_raw]
x_raw = [a.replace("really", "") for a in x_raw]
x_raw = [a.replace("like", "") for a in x_raw]
x_raw = [a.replace("product", "") for a in x_raw]

In [201]:
data[['DOC_ID', 'PRODUCT_CATEGORY', 'PRODUCT_TITLE', 'REVIEW_TITLE', 'REVIEW_TEXT']].head()

DOC_ID  ...                                        REVIEW_TEXT
0       1  ...  When least you think so, this product will sav...
1       2  ...  Lithium batteries are something new introduced...
2       3  ...  I purchased this swing for my baby. She is 6 m...
3       4  ...  I was looking for an inexpensive desk calcolat...
4       5  ...  I only use it twice a week and the results are...

[5 rows x 5 columns]

**Preprocess**

In [0]:
#Preprocess corpus
import string
from string import punctuation
from nltk.corpus import stopwords

def preprocess(text):
    # word tokenisation, including punctuation removal
    tokenizer = RegexpTokenizer(r'\w+')

    # lowercasing
    text = [t.lower() for t in text]

    # stopword removal
    stop = set(stopwords.words('english'))
    text = [t for t in text if t not in stop]
    
    # lemmatisation
    lemmatiser = WordNetLemmatizer()
    text = [lemmatiser.lemmatize(t) for t in text]

    # remove numbers and empty space
    digits = ' 0123456789'
    text = [t for t in text if t not in digits]

    #tokenize sentences
    for sent in text:
        yield(gensim.utils.simple_preprocess(str(sent)))
        
    return text

**Ngrams**

In [0]:
# Use bigrams to improve model
def create_bigrams(text):
    bigram = gensim.models.Phrases(text, min_count=4, threshold=40)


**Define Model and parameters**

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

model = ['LDA', 'NMF']
vectorizer = ['count', 'tfidf']

def topic_model(data, model, vectorizer, n_topics=10):
    """
    model: method for topic model
    vectorizer: method for representation of words
    n_topics = number of topics
    """

    if vectorizer == 'count':
        vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')
        x_train = vect.fit_transform(x)

    if vectorizer == 'tfidf':
        vect = TfidfVectorizer(max_df=0.8, min_df=2, stop_words='english')
        x_train = vect.fit_transform(x)

    if model == 'LDA':
        LDA = LatentDirichletAllocation(n_components=n_topics, learning_decay=0.5, random_state=42)
        _model = LDA.fit(x_train)
    
    if model == 'NMF':
        nmf = NMF(n_components=n_topics, random_state=42)
        _model = nmf.fit(x_train)

    return _model, vect

**Prepare data to train**

In [205]:
x = list(preprocess(x_raw))
x = [bigram[line] for line in x]
x = [' '.join(i) for i in x]

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [0]:
LDA, vect = topic_model(x, model='LDA', vectorizer='tfidf')
#LDA.get_params()
nmf, vect = topic_model(x, model='NMF', vectorizer='tfidf')
#nmf.get_params()

**Show topics infered**

In [247]:
def create_topics_table(model, vectorizer, n_topics):
    keywords = np.array(vectorizer.get_feature_names())
    topics = []
    for weights in model.components_:
        top_index = (-weights).argsort()[:n_topics]
        topic = keywords.take(top_index)
        w = np.sort(-weights)[:5]
        w = w.round(1)
        w = w.astype(str)
        ind = map(' '.join, zip(topic, w))
        topics.append(ind)
        #topics.append(keywords.take(top_keywords))
    return topics

print('Top 10 words for each topic with LDA')
topic_keywords = create_topics_table(model=LDA, vectorizer=vect, n_topics=5)       
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
display(df_topic_keywords)

print('Top 10 words for each topic with NMF')
topic_keywords2 = create_topics_table(model=nmf, vectorizer=vect, n_topics=5)
df_topic_keywords2 = pd.DataFrame(topic_keywords2)
df_topic_keywords2.columns = ['Word '+str(i) for i in range(df_topic_keywords2.shape[1])]
df_topic_keywords2.index = ['Topic '+str(i) for i in range(df_topic_keywords2.shape[0])]
display(df_topic_keywords2)



Top 10 words for each topic with LDA


Word 0         Word 1  ...          Word 3            Word 4
Topic 0        ítem -4.9     zumba -4.8  ...         zs -3.5           zr -3.5
Topic 1  glad_found -5.3    roller -4.7  ...         ab -3.8  thermometer -3.7
Topic 2        love -6.7      book -6.6  ...     ie_utf -4.9          war -4.8
Topic 3      movie -61.9     book -49.7  ...      read -22.9      movies -16.6
Topic 4       love -55.7      bag -52.5  ...      nice -45.7        wear -45.7
Topic 5     price -215.3    easy -208.4  ...   bought -193.8       time -191.4
Topic 6       skin -29.1  printer -24.4  ...     smell -21.0        love -20.0
Topic 7      knife -22.8      dog -20.0  ...       fit -14.4     carrier -11.4
Topic 8       love -23.3     ring -15.8  ...  necklace -11.9       great -11.2
Topic 9     coffee -44.4     love -33.6  ...      hair -23.0        best -22.7

[10 rows x 5 columns]

Top 10 words for each topic with NMF


Word 0        Word 1  ...           Word 3        Word 4
Topic 0   work -1.0     beca -0.9  ...         got -0.8      don -0.7
Topic 1  watch -2.7    looks -0.4  ...        time -0.3  watches -0.2
Topic 2  price -1.7  quality -1.7  ...        size -0.8    great -0.8
Topic 3   love -2.5    color -0.2  ...     perfect -0.2     kids -0.1
Topic 4    bag -2.8  leather -0.4  ...        bags -0.3   inside -0.2
Topic 5   easy -2.5    works -0.7  ...       light -0.4    clean -0.3
Topic 6     tv -2.6  picture -0.8  ...      screen -0.3   remote -0.2
Topic 7   case -2.1    phone -1.5  ...  protection -0.2     ipad -0.2
Topic 8   game -2.2     play -0.7  ...       games -0.5     kids -0.3
Topic 9   book -1.7     read -0.7  ...       books -0.2    story -0.2

[10 rows x 5 columns]

In [250]:
!pip install pyLDAvis
from pyLDAvis import sklearn as sklearn_lda
import os
import pickle 
import pyLDAvis
LDAvis_data_filepath = os.path.join('./ldavis_prepared_'+str(10))

if 1 == 1:
    x_train = vect.fit_transform(x)
    LDAvis_prepared = sklearn_lda.prepare(LDA, x_train, vect)
with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

with open(LDAvis_data_filepath, 'rb') as f:
     LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './ldavis_prepared_'+ str(10) +'.html')

pyLDAvis.display(LDAvis_prepared)

**Show reviews and topic assigned**

In [0]:
# Topic ID for each review in dataset: review ID, review text, topic ID

def show_results(model, vectorizer):
    x_train = vectorizer.fit_transform(x)
    topic_values = model.transform(x_train)
    data['Topic'] = topic_values.argmax(axis=1)
    #data[['REVIEW_TEXT', 'Topic']]

    result = data[['REVIEW_TEXT', 'Topic']][:50]
    from IPython.display import display, HTML

    display(HTML(result.to_html()))

In [252]:
lda_result = show_results(LDA, vect)

In [253]:
nmf_result = show_results(nmf, vect)